In [1]:
#Feature extraction of acoustics index

from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display
from time import time #importamos la función time para capturar tiempos

import sklearn
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')
import soundfile as sf
from Indices import *
import glob

In [22]:
import glob
glob.glob('/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/'+'*.wav')

['/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151212_123000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151213_185000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151215_011000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151216_073000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151222_135000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151223_201000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151212_032000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151212_033000.wav',
 '/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2015/3788/20151212/MIRAMEDI3788_20151212_03400

In [ ]:
header = 'filename chroma_cqt  chroma_cens spec_contrast spectral_flatness tonnetz'

header = header.split()

header += ['label']

file = open('data2sepnewfeatures.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'{g}'):
        clear_output()
        print(g)
        print(filename)
        songname = f'{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
        spec_contrast = librosa.feature.spectral_contrast(y=y)
        spectral_flatness = librosa.feature.spectral_flatness(y=y)
        tonnetz = librosa.feature.tonnetz(y)

        
        to_append = f'{filename} {np.mean(chroma_cqt)} {np.mean(chroma_cens)} {np.mean(spec_contrast)} {np.mean(spectral_flatness)} {np.mean(tonnetz)}'    
        to_append += f' {g}'
        file = open('data2sepnewfeatures.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
titulos = ["ADI", "ACItf", "BI", "TE", "ESM", "NDSI", "P", "M", "MID" "BNT", "MD",
                       "FM", "SF", "RMS"]
header = 'filename chroma_stft  spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

header = header.split()

header += titulos
header += ['label']
##Variables para indices
tipo_ventana = "hann"
sobreposicion = 0
tamano_ventana = 512
nfft= 512
bio_band = (2000, 8000)
tech_band = (200, 1500)

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'{g}'):
        clear_output()
        print(g)
        print(filename)
        songname = f'{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)

        print(spec_bw)
        x, Fs = sf.read(songname)
        audio = x
        nmin = len(audio) // (60 * Fs)

        ## indices acusticos
        f, t, s = signal.spectrogram(audio, Fs, window=tipo_ventana, nperseg=nmin * tamano_ventana,
                                 mode="magnitude", \
                                 noverlap=sobreposicion, nfft=nmin * nfft)
        nmin = len(audio) // (60 * Fs)


        adi=ADI(s, 10000, 1000, -50)
        print(adi)
        aci=ACItf(audio, Fs, 5, s)
       # beta=beta(s, f, bio_band)//nmin
        te=temporal_entropy(audio, Fs)
        sme=spectral_maxima_entropy(s, f, 482, 8820)
        ndsi=NDSI(s, f, bio_band, tech_band)
        r=rho(s, f, bio_band, tech_band)
        me=median_envelope(audio, Fs, 16)
        #number_of_peaks(s, f, 10 * nmin)
        mba=mid_band_activity(s, f, 450, 3500)
        #np.mean(background_noise_freq(s))
        bnt=background_noise_time(wav2SPL(audio, -11, 9, 0.707), 5)
        md=musicality_degree(audio, Fs, tamano_ventana, nfft, tipo_ventana, sobreposicion)
        fm=frequency_modulation(s)
        we=wiener_entropy(audio, tamano_ventana, nfft, tipo_ventana, sobreposicion)
        #rms=rms(audio)

        
        
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {adi} {aci} {te} {sme} {ndsi} {r} {me} {mba} {bnt} {md} {fm} {we}'    
       # to_append += {adi} {aci} {beta}{te}{sme}{ndsi}{r}{me}{mba}{bnt}{md}{fm}{we}{rms}
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [2]:
%cd /media/david/Seagate Expansion Drive/Bioacustica/Caribe/2016/

import os
os.getcwd()
genres=glob.glob(f''+'*')
genres

/media/david/Seagate Expansion Drive/Bioacustica/Caribe/2016


['302143', '302151', '3786', '3787', '5067', '5069', '5070', '5071', '5072']

In [5]:
genres[3:]

['5072', 'JSC5069', 'LAS40-5069']

In [3]:
titulos = ["ADI", "TE","EMS" , "MD", "p"]
header = 'filename chroma_stft  spectral_centroid spectral_bandwidth tonnets '

header = header.split()

header += titulos
header += ['label']
header

['filename',
 'chroma_stft',
 'spectral_centroid',
 'spectral_bandwidth',
 'tonnets',
 'ADI',
 'TE',
 'EMS',
 'MD',
 'p',
 'label']

In [14]:
g='5072'
glob.glob(g+'/*')

['5072/20151212']

In [15]:
fecha=glob.glob(g+'/*')

In [19]:
file = open(fecha+'variablesprueba4sep.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

TypeError: can only concatenate list (not "str") to list

In [ ]:

##Variables para indices
tipo_ventana = "hann"
sobreposicion = 0
tamano_ventana = 512
nfft= 512
bio_band = (2000, 8000)
tech_band = (200, 1500)



'''Gneres es la grabadora, fecha es la fecha dentro de la grabadora y filename es el nombre de la grabacion'''

for g in genres[5:]:
    clear_output()
    print(g+'+++++++++++++++++++++++++++++++++++++++++++++          ')
    for fecha in glob.glob(g+'/*'):
        print(fecha+'********************************************')
        file = open(fecha+'variablesprueba4sep.csv', 'w', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(header)
        for filename in glob.glob(fecha+'/*.wav'):
            try:
                print(filename)

                songname = filename
                y, sr = librosa.load(songname, mono=True, duration=30)
                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                tonnetz = librosa.feature.tonnetz(y)

                x, Fs = sf.read(songname)
                audio = x
                nmin = len(audio) // (60 * Fs)
                ## indices acusticos
                f, t, s = signal.spectrogram(audio, Fs, window=tipo_ventana, nperseg=nmin * tamano_ventana,
                                         mode="magnitude", \
                                         noverlap=sobreposicion, nfft=nmin * nfft)
                nmin = len(audio) // (60 * Fs)


                adi=ADI(s, 10000, 1000, -50)
                te=temporal_entropy(audio, Fs)
                sme=spectral_maxima_entropy(s, f, 482, 8820)
                md=musicality_degree(audio, Fs, tamano_ventana, nfft, tipo_ventana, sobreposicion)
                r=rho(s, f, bio_band, tech_band)


                to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(tonnetz)} {adi} {te} {sme} {md} {r}'    
               # to_append += {adi} {aci} {beta}{te}{sme}{ndsi}{r}{me}{mba}{bnt}{md}{fm}{we}{rms}
                to_append += f' {g}'
                file = open(fecha+'variablesprueba4sep.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                
            except:
                print("grabacion con error:"+filename)
                

5070+++++++++++++++++++++++++++++++++++++++++++++          
5070/20160214********************************************
5070/20160214/GUAM5070__0__20160214_173000.wav
5070/20160214/GUAM5070__0__20160215_235000.wav
5070/20160214/GUAM5070__0__20160217_061000.wav
5070/20160214/GUAM5070__0__20160224_123000.wav
5070/20160214/GUAM5070__0__20160225_185000.wav
5070/20160214/GUAM5070__0__20160227_011000.wav
5070/20160214/GUAM5070__0__20160305_073000.wav
5070/20160214/GUAM5070__0__20160306_134000.wav
5070/20160214/GUAM5070__0__20160307_201000.wav
5070/20160214/GUAM5070__0__20160214_012000.wav
5070/20160214/GUAM5070__0__20160214_013000.wav
5070/20160214/GUAM5070__0__20160214_014000.wav
5070/20160214/GUAM5070__0__20160214_015000.wav
5070/20160214/GUAM5070__0__20160214_020000.wav
5070/20160214/GUAM5070__0__20160214_021000.wav
5070/20160214/GUAM5070__0__20160214_022000.wav
5070/20160214/GUAM5070__0__20160214_023000.wav
5070/20160214/GUAM5070__0__20160214_024000.wav
5070/20160214/GUAM5070__0__20160214_

5070/20160214/GUAM5070__0__20160215_063000.wav
5070/20160214/GUAM5070__0__20160215_064000.wav
5070/20160214/GUAM5070__0__20160215_065000.wav
5070/20160214/GUAM5070__0__20160215_070000.wav
5070/20160214/GUAM5070__0__20160215_071000.wav
5070/20160214/GUAM5070__0__20160215_072000.wav
5070/20160214/GUAM5070__0__20160215_074000.wav
5070/20160214/GUAM5070__0__20160215_075000.wav
5070/20160214/GUAM5070__0__20160215_080000.wav
5070/20160214/GUAM5070__0__20160215_081000.wav
5070/20160214/GUAM5070__0__20160215_082000.wav
5070/20160214/GUAM5070__0__20160215_083000.wav
5070/20160214/GUAM5070__0__20160215_084000.wav
5070/20160214/GUAM5070__0__20160215_085000.wav
5070/20160214/GUAM5070__0__20160215_090000.wav
5070/20160214/GUAM5070__0__20160215_091000.wav
5070/20160214/GUAM5070__0__20160215_092000.wav
5070/20160214/GUAM5070__0__20160215_093000.wav
5070/20160214/GUAM5070__0__20160215_094000.wav
5070/20160214/GUAM5070__0__20160215_100000.wav
5070/20160214/GUAM5070__0__20160215_101000.wav
5070/20160214

5070/20160214/GUAM5070__0__20160216_115000.wav
5070/20160214/GUAM5070__0__20160216_120000.wav
5070/20160214/GUAM5070__0__20160216_121000.wav
5070/20160214/GUAM5070__0__20160216_122000.wav
5070/20160214/GUAM5070__0__20160216_123000.wav
5070/20160214/GUAM5070__0__20160216_124000.wav
5070/20160214/GUAM5070__0__20160216_125000.wav
5070/20160214/GUAM5070__0__20160216_130000.wav
5070/20160214/GUAM5070__0__20160216_131000.wav
5070/20160214/GUAM5070__0__20160216_132000.wav
5070/20160214/GUAM5070__0__20160216_133000.wav
5070/20160214/GUAM5070__0__20160216_134000.wav
5070/20160214/GUAM5070__0__20160216_140000.wav
5070/20160214/GUAM5070__0__20160216_141000.wav
5070/20160214/GUAM5070__0__20160216_142000.wav
5070/20160214/GUAM5070__0__20160216_143000.wav
5070/20160214/GUAM5070__0__20160216_144000.wav
5070/20160214/GUAM5070__0__20160216_145000.wav
5070/20160214/GUAM5070__0__20160216_150000.wav
5070/20160214/GUAM5070__0__20160216_151000.wav
5070/20160214/GUAM5070__0__20160216_152000.wav
5070/20160214

5070/20160214/GUAM5070__0__20160217_170000.wav
5070/20160214/GUAM5070__0__20160217_171000.wav
5070/20160214/GUAM5070__0__20160217_172000.wav
5070/20160214/GUAM5070__0__20160217_173000.wav
5070/20160214/GUAM5070__0__20160217_174000.wav
5070/20160214/GUAM5070__0__20160217_180000.wav
5070/20160214/GUAM5070__0__20160217_181000.wav
5070/20160214/GUAM5070__0__20160217_182000.wav
5070/20160214/GUAM5070__0__20160217_183000.wav
5070/20160214/GUAM5070__0__20160217_184000.wav
5070/20160214/GUAM5070__0__20160217_185000.wav
5070/20160214/GUAM5070__0__20160217_190000.wav
5070/20160214/GUAM5070__0__20160217_191000.wav
5070/20160214/GUAM5070__0__20160217_192000.wav
5070/20160214/GUAM5070__0__20160217_193000.wav
5070/20160214/GUAM5070__0__20160217_194000.wav
5070/20160214/GUAM5070__0__20160217_195000.wav
5070/20160214/GUAM5070__0__20160217_200000.wav
5070/20160214/GUAM5070__0__20160217_202000.wav
5070/20160214/GUAM5070__0__20160217_203000.wav
5070/20160214/GUAM5070__0__20160217_204000.wav
5070/20160214

5070/20160214/GUAM5070__0__20160224_222000.wav
5070/20160214/GUAM5070__0__20160224_223000.wav
5070/20160214/GUAM5070__0__20160224_224000.wav
5070/20160214/GUAM5070__0__20160224_225000.wav
5070/20160214/GUAM5070__0__20160224_230000.wav
5070/20160214/GUAM5070__0__20160224_231000.wav
5070/20160214/GUAM5070__0__20160224_232000.wav
5070/20160214/GUAM5070__0__20160224_233000.wav
5070/20160214/GUAM5070__0__20160224_234000.wav
5070/20160214/GUAM5070__0__20160224_235000.wav
5070/20160214/GUAM5070__0__20160225_000000.wav
5070/20160214/GUAM5070__0__20160225_002000.wav
5070/20160214/GUAM5070__0__20160225_003000.wav
5070/20160214/GUAM5070__0__20160225_004000.wav
5070/20160214/GUAM5070__0__20160225_005000.wav
5070/20160214/GUAM5070__0__20160225_010000.wav
5070/20160214/GUAM5070__0__20160225_011000.wav
5070/20160214/GUAM5070__0__20160225_012000.wav
5070/20160214/GUAM5070__0__20160225_013000.wav
5070/20160214/GUAM5070__0__20160225_014000.wav
5070/20160214/GUAM5070__0__20160225_015000.wav
5070/20160214

5070/20160214/GUAM5070__0__20160226_033000.wav
5070/20160214/GUAM5070__0__20160226_034000.wav
5070/20160214/GUAM5070__0__20160226_035000.wav
5070/20160214/GUAM5070__0__20160226_040000.wav
5070/20160214/GUAM5070__0__20160226_042000.wav
5070/20160214/GUAM5070__0__20160226_043000.wav
5070/20160214/GUAM5070__0__20160226_044000.wav
5070/20160214/GUAM5070__0__20160226_045000.wav
5070/20160214/GUAM5070__0__20160226_050000.wav
5070/20160214/GUAM5070__0__20160226_051000.wav
5070/20160214/GUAM5070__0__20160226_052000.wav
5070/20160214/GUAM5070__0__20160226_053000.wav
5070/20160214/GUAM5070__0__20160226_054000.wav
5070/20160214/GUAM5070__0__20160226_055000.wav
5070/20160214/GUAM5070__0__20160226_060000.wav
5070/20160214/GUAM5070__0__20160226_061000.wav
5070/20160214/GUAM5070__0__20160226_062000.wav
5070/20160214/GUAM5070__0__20160225_211000.wav
5070/20160214/GUAM5070__0__20160225_233000.wav
5070/20160214/GUAM5070__0__20160226_015000.wav
5070/20160214/GUAM5070__0__20160226_041000.wav
5070/20160214

5070/20160214/GUAM5070__0__20160227_073000.wav
5070/20160214/GUAM5070__0__20160227_074000.wav
5070/20160214/GUAM5070__0__20160227_075000.wav
5070/20160214/GUAM5070__0__20160227_080000.wav
5070/20160214/GUAM5070__0__20160227_082000.wav
5070/20160214/GUAM5070__0__20160227_083000.wav
5070/20160214/GUAM5070__0__20160227_084000.wav
5070/20160214/GUAM5070__0__20160227_085000.wav
5070/20160214/GUAM5070__0__20160227_090000.wav
5070/20160214/GUAM5070__0__20160227_091000.wav
5070/20160214/GUAM5070__0__20160227_092000.wav
5070/20160214/GUAM5070__0__20160227_093000.wav
5070/20160214/GUAM5070__0__20160227_094000.wav
5070/20160214/GUAM5070__0__20160227_095000.wav
5070/20160214/GUAM5070__0__20160227_100000.wav
5070/20160214/GUAM5070__0__20160227_101000.wav
5070/20160214/GUAM5070__0__20160227_102000.wav
5070/20160214/GUAM5070__0__20160227_104000.wav
5070/20160214/GUAM5070__0__20160227_105000.wav
5070/20160214/GUAM5070__0__20160227_110000.wav
5070/20160214/GUAM5070__0__20160227_111000.wav
5070/20160214

5070/20160214/GUAM5070__0__20160305_165000.wav
5070/20160214/GUAM5070__0__20160305_191000.wav
5070/20160214/GUAM5070__0__20160305_213000.wav
5070/20160214/GUAM5070__0__20160305_235000.wav
5070/20160214/GUAM5070__0__20160306_021000.wav
5070/20160214/GUAM5070__0__20160306_043000.wav
5070/20160214/GUAM5070__0__20160306_065000.wav
5070/20160214/GUAM5070__0__20160306_091000.wav
5070/20160214/GUAM5070__0__20160306_113000.wav
5070/20160214/GUAM5070__0__20160305_122000.wav
5070/20160214/GUAM5070__0__20160305_123000.wav
5070/20160214/GUAM5070__0__20160305_124000.wav
5070/20160214/GUAM5070__0__20160305_125000.wav
5070/20160214/GUAM5070__0__20160305_130000.wav
5070/20160214/GUAM5070__0__20160305_131000.wav
5070/20160214/GUAM5070__0__20160305_132000.wav
5070/20160214/GUAM5070__0__20160305_133000.wav
5070/20160214/GUAM5070__0__20160305_134000.wav
5070/20160214/GUAM5070__0__20160305_135000.wav
5070/20160214/GUAM5070__0__20160305_140000.wav
5070/20160214/GUAM5070__0__20160305_141000.wav
5070/20160214

In [ ]:
g